# BAYER - VIGILOO --> Control de Interfaces y Consistencia de DATOS

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import json

def obtener_enlaces(url):
    # Realizar la petición HTTP a la página web
    respuesta = requests.get(url)
    # Si la petición fue exitosa, procesar el contenido
    if respuesta.status_code == 200:
        soup = BeautifulSoup(respuesta.content, 'html.parser')
        # Extraer todos los elementos <a> de la página
        enlaces = soup.find_all('a', href=True)
        # Crear una lista para almacenar los datos
        datos = []
        # Recorrer cada enlace y almacenar la información necesaria
        for enlace in enlaces:
            # Extraer el href del enlace
            href = enlace['href']
            # Extraer el texto del enlace, que es el nombre del archivo
            texto = enlace.text
            # Intentar extraer las observaciones que están en el contenido antes del enlace
            # Esto asume que el texto de observaciones está justo antes del enlace
            observaciones = enlace.previous_sibling
            if observaciones:
                observaciones = observaciones.strip()  #Remueve los espacios al principio y al final de la línea
                # Dividir las observaciones en partes
                partes = observaciones.split("   ")
                #print(partes)
                # Extraer la fecha y la hora (primeras dos partes)
                fecha_hora = ' '.join(partes[:1])
                #print(fecha_hora)
                # Extraer el tamaño (última parte)
                tamaño = partes[-1]
                #print(tamaño)
            else:
                fecha_hora = "No disponible"
                tamaño = "No disponible"
            # Añadir a la lista como una tupla
            datos.append((texto, "http://rtv-b2b-api-logs.vigiloo.net" + href, fecha_hora, tamaño))    # datos.append((texto, url + href, fecha_hora, tamaño))
        # Convertir la lista en un DataFrame
        df = pd.DataFrame(datos, columns=['NombreArchivo', 'URL', 'FechaHora', 'Tamaño'])
        df.drop(0,axis=0)
        # Convertir la columna 'FechaHora' a tipo datetime
        df['FechaHora'] = pd.to_datetime(df['FechaHora'], dayfirst=False, format='%m/%d/%Y %I:%M %p',errors='coerce') 
        # Convertir la columna 'Tamaño' a tipo numérico
        df['Tamaño'] = pd.to_numeric(df['Tamaño'], errors='coerce')
        df['Embarque'] = df['NombreArchivo'].str.extract(r'-(\d+)\.log')
        df.drop(0,axis=0, inplace=True)  # Borro la Primera línea que apunta al directorio padre        
        return df
    else:
        return "Error al acceder a la página"

### 1) Dataframe con el TOTAL de los archivos existentes de archivos de Log

In [2]:
# URL de la página de donde se desean extraer los enlaces
url = "http://rtv-b2b-api-logs.vigiloo.net/LoadUpload/"
archivos_df = obtener_enlaces(url)
if isinstance(archivos_df, pd.DataFrame):
   print(archivos_df.head(1))
else:
   print("Error")

In [3]:
# FILTRAR Archivos_df y generar subset en archivo SALIDA
# Inicio Nueva Nomenclatura de Archivos 01/18/2024

fecha_inicio = datetime.datetime(year=2025,month=1,day=1)

entrada = archivos_df[(archivos_df['FechaHora'] >=fecha_inicio)]
#salida.dropna()
# print(filtered_df)


In [ ]:
entrada.info()

In [4]:
# FILTRADO y ORDENADO del dataframe para procesar secuencialmente

graficar = entrada.sort_values(by=['Embarque', 'FechaHora'])

In [5]:
# Recuento de la Cantidad de Archvios por Minuto
agruparD = graficar['FechaHora'].dt.floor('D').value_counts()

#agruparD = graficar['FechaHora'].dt.floor('min').value_counts()

In [6]:
#agruparD["Fecha"] = [date.strftime('%Y-%m-%d') for date in  agruparD.index]
agruparD['Fecha'] = pd.to_datetime(agruparD.index, format = '%Y-%m-%d')

In [ ]:
agruparD.info()

In [ ]:
print(graficar.head())
print(graficar.info())


In [ ]:
print(graficar.dtypes)


In [ ]:
print(agruparD)


In [7]:
import matplotlib.pyplot as plt
import pandas as pd

# Verificar si la columna FechaHora es válida
if 'FechaHora' not in graficar.columns:
    raise ValueError("La columna 'FechaHora' no existe en el DataFrame.")

# Convertir a datetime si es necesario
if not pd.api.types.is_datetime64_any_dtype(graficar['FechaHora']):
    graficar['FechaHora'] = pd.to_datetime(graficar['FechaHora'], errors='coerce')

# Filtrar filas con fechas no válidas
graficar = graficar.dropna(subset=['FechaHora'])

# Agrupar por día y contar archivos
agruparD = graficar['FechaHora'].dt.floor('D').value_counts().sort_index()

# Verificar si hay datos para graficar
if agruparD.empty:
    print("⚠️ No hay datos disponibles para graficar.")
else:
    # Crear figura
    plt.figure(figsize=(10, 5))  # Tamaño más amplio

    # Graficar como barra
    agruparD.plot(kind='bar', color='royalblue')

    # Configuración de etiquetas
    plt.title('Archivos recibidos por Día')
    plt.xlabel('Día')
    plt.ylabel('Cantidad de Archivos')

    # Rotar etiquetas del eje X
    plt.xticks(rotation=65, ha='right')

    # Mostrar gráfico
    plt.show()



## Explicación del Formato

%m: Representa el mes como un número con dos dígitos (01 a 12).

%d: Representa el día del mes como un número con dos dígitos (01 a 31).

%Y: Representa el año con cuatro dígitos.

%I: Representa la hora en formato de 12 horas (01 a 12).

%M: Representa los minutos.

%p: Representa AM o PM.

En nuestro caso como la fecha tiene los Dias y los Meses sin PADD hay que utilizar la siguiente opción

### pd.to_datetime(salida['FechaHora'], dayfirst=False).dt.strftime('%m/%d/%Y %I:%M %p')


https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

https://pandas.pydata.org/pandas-docs/version/1.0.1/reference/api/pandas.to_datetime.html

https://github.com/pandas-dev/pandas/issues/3341

In Linux "#" is replaced by "-":
%-d, %-H, %-I, %-j, %-m, %-M, %-S, %-U, %-w, %-W, %-y, %-Y
In Windows "-" is replaced by "#":
%#d, %#H, %#I, %#j, %#m, %#M, %#S, %#U, %#w, %#W, %#y, %#Y

#Linux
mydatetime.strftime('%-m/%d/%Y %-I:%M%p')
#Windows
mydatetime.strftime('%#m/%d/%Y %#I:%M%p')
Source: https://msdn.microsoft.com/en-us/library/fe06s4ak.aspx

## Procesar un registro JSON

* https://brightdata.es/blog/procedimientos/parse-json-data-with-python
* https://docs.python.org/3/tutorial/datastructures.html#dictionaries
* https://docs.python.org/3/tutorial/datastructures.html#more-on-lists


In [16]:
# Busco un registro para probar

reg=2
url = entrada.iloc[reg]['URL']

print(url)
print('Archivo : ',entrada.iloc[reg]['NombreArchivo'])
print('Embarque: ',entrada.iloc[reg]['Embarque'])
print('Fecha   : ',entrada.iloc[reg]['FechaHora'])
print('Tamaño  : ',entrada.iloc[reg]['Tamaño'])
embarque = pd.read_json(url)

### Ejemplo de Uso

https://stackoverflow.com/questions/74118717/create-new-dataframe-with-json-objects-from-another-dataframe#:~:text=Solution%20using%20pandas&text=It%20provides%20a%20DataFrame%20object,dataframe%20with%20the%20original%20one



## Procesar El archivo SALIDA identificando los JSON

### PARAMETROS para la CONSULTA.

#### Comienza el proceso

In [34]:
# Extraer valores únicos de la columna 'Embarque' para usar en Query SI QUIERO PROCESAR TODO
embarques = entrada['Embarque'].unique()

# Formatear los valores entre apóstrofes y separados por comas
#REGISTROS = ', '.join(f"'{embarque}'" for embarque in embarques)

REGISTROS = ','.join(f"{embarque}" for embarque in embarques)

REGISTROS = tuple(REGISTROS.split(','))


In [14]:
print(REGISTROS)

In [13]:
type(REGISTROS)

In [8]:
# PARAMETROS SI QUIERO CONSULTAS REGISTROS RECIBIDOSA EN EL ENVÍO --> SOBRE ESCRIBIR REGISTROS

REGISTROS=('834879704',
'834879927',
'834879946',
'834879970',
'834880047',
'834880418',
'834880455',
'834880469'
)

In [10]:
# FUNCIONES PARA LEER ARCHIVOS JSON
import requests
import json

def procesar_entregas_desde_url(url, modo='imprimir'):
    # Descargar el contenido del archivo JSON desde la URL
    respuesta = requests.get(url)
    respuesta.raise_for_status()  # Esto lanzará una excepción si ocurre un error

    # Leer el contenido JSON
    datos = respuesta.json()    
    resultados = []
    
    # Leer Datos de la Cabecera    
    resultado = f"\nOrigen: {datos['LogisticGroupCode']},  Embarque: {datos['Id']}, Date: {datos['Date']}, Carrier: {datos['CarrierName']}\n"
    resultados.append(resultado)
    # Imprimir la información si el modo es 'imprimir'
    if modo == 'imprimir':
        print(resultado)

    # Procesar y almacenar la información en una lista
    for stop in datos['StopList']:
        # Verificar que la parada tenga entregas
        if stop['OperationType'] == 'Dropped' and stop.get('Deliveries'):
            for delivery in stop['Deliveries']:
                delivery_number = delivery['DeliveryNumber']
                location = stop['LocationName']
                for item in delivery['Items']:
                    material_name = item['MaterialName']
                    batch = item.get('Batch', 'N/A')
                    qty = item['Qty']
                    
                    resultado = f"   DeliveryNumber: {delivery_number}, LocationName: {location}, MaterialName: {material_name}, Batch: {batch}, Qty: {qty}"
                    resultados.append(resultado)

                    # Imprimir la información si el modo es 'imprimir'
                    if modo == 'imprimir':
                        print(resultado)
    
    # Devolver los resultados si el modo es 'devolver'
    if modo == 'devolver':
        return resultados

    # Almacenar los resultados en un archivo si el modo es 'almacenar'
    if modo == 'almacenar':
        with open('resultados.txt', 'a', encoding="utf-8") as archivo_salida:
            for resultado in resultados:
                archivo_salida.write(resultado + '\n')
            archivo_salida.close()
                


# Ejemplo para validar el uso de la función
# url = 'http://rtv-b2b-api-logs.vigiloo.net/LoadUpload/638508724764465417-834195574.log'
# procesar_entregas_desde_url(url, modo='imprimir')


#### Explicación
* Leer el archivo JSON: Utilizamos json.load() para cargar el contenido del archivo JSON en una variable datos.
* Procesar las entregas: Iteramos sobre la lista de paradas (StopList). Para cada parada, obtenemos la lista de entregas (Deliveries).
* Extraer y mostrar la información: Para cada entrega, extraemos DeliveryNumber, LocationName, MaterialName, Batch, y Qty, y luego imprimimos esta información en el formato deseado.
* Función versátil: La función procesar_entregas permite imprimir, devolver o almacenar la información según el modo especificado.

##### Ejemplo de uso de la función
procesar_entregas('archivo.json', modo='imprimir')

Utiliza los PARAMETROS ingresados anteriormente

In [11]:
# FILTRADO y ORDENADO del dataframe para procesar secuencialmente

resumen = entrada.loc[entrada["Embarque"].isin(REGISTROS)]
salida = resumen.sort_values(by=['Embarque', 'FechaHora'])

In [ ]:
salida.info()


In [12]:
# Recuento de la Cantidad de Archvios por Minuto
grouped = salida['FechaHora'].dt.floor('min').value_counts()

In [13]:
import matplotlib.pyplot as plt

# Creación del gráfico
plt.figure(figsize=(8, 3)) # Aumenta el ancho aquí
grouped.plot(kind='bar') 
plt.title('Files received per minute') 
plt.xlabel('Minutos') 
plt.ylabel('Cantidad de Archivos')
plt.xticks(rotation=65, ha='right')  #Rotar el ángulo de las etiquetas x
plt.show()


In [14]:
reenvios = salida['Embarque'].value_counts()

# Creación del gráfico
plt.figure(figsize=(8, 3)) # Aumenta el ancho aquí
reenvios.plot(kind='bar', width=0.5) # Ajuste la anchura de las barras aquí
plt.title('Reenvios del Mismo Embarque')
plt.xlabel('Embarque')
plt.ylabel('Cantidad de Veces')
plt.xticks(rotation=65, ha='right')

plt.show()


In [15]:
reenvios = salida['Embarque'].value_counts()

# Creación del gráfico
plt.figure(figsize=(8, 3))  # Ajusta el tamaño del gráfico
reenvios.plot(kind='bar', width=0.5, color='skyblue')  # Color opcional

# Agregar título y etiquetas
plt.title('Reenvíos del Mismo Embarque')
plt.xlabel('Embarque')
plt.ylabel('Cantidad de Veces')
plt.xticks(rotation=65, ha='right')

# Calcular el total de embarques
total_embarques = reenvios.sum()

# Posicionar el label flotante en el centro del gráfico
plt.text(
    len(reenvios) / 2,  # Posición horizontal en el centro
    reenvios.max() / 2,  # Posición vertical en el centro de las barras
    f'Total: {total_embarques}', 
    fontsize=10, color='black', fontweight='bold',
    ha='center', va='center', bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5')
)

# Mostrar el gráfico
plt.show()


In [ ]:
# RUTINA PARA IMPRIMIR SALIDAS por la Terminal

for i in range(len(salida)):
    if salida.iloc[i]['Embarque'] in (REGISTROS):
        print('----------------------------------------------------------------')
        print('Embarque: ',salida.iloc[i]['Embarque'])
        url = salida.iloc[i]['URL']
        print(f'Archivo: ',salida.iloc[i]['FechaHora'],url)
        # Procesar cada Embarque
        resultado = procesar_entregas_desde_url(url, modo='imprimir')


## ANÁLISIS SOLO DE LOS FALTANTES o INCOMPLETOS

In [17]:
# PARAMETROS SI QUIERO CONSULTAS REGISTROS RECIBIDOSA EN EL ENVÍO --> SOBRE ESCRIBIR REGISTROS

FALTANTES=('834879704',
'834879927',
'834879946',
'834879970',
'834880047',
'834880418',
'834880455',
'834880469'
)

In [18]:
# FILTRADO y ORDENADO del dataframe para procesar secuencialmente

faltan = entrada.loc[entrada["Embarque"].isin(FALTANTES)]
fallaron = faltan.sort_values(by=['Embarque', 'FechaHora'])

In [19]:
# Recuento de la Cantidad de Archvios por Minuto
grouped = fallaron['FechaHora'].dt.floor('min').value_counts()

reenvios = fallaron['Embarque'].value_counts()

# Creación del gráfico
plt.figure(figsize=(4, 3)) # Aumenta el ancho aquí
reenvios.plot(kind='bar', width=0.5) # Ajuste la anchura de las barras aquí
plt.title('Reenvios de Archivos con Embarque Incompletos')
plt.xlabel('Embarque')
plt.ylabel('Cantidad de Veces')
plt.xticks(rotation=65, ha='right')

plt.show()


In [20]:
# DETALLE de los ARCHIVOS JSON con FALTANTES
# import requests
# import json

# RUTINA PARA IMPRIMIR SALIDAS por la Terminal

for i in range(len(salida)):
    if salida.iloc[i]['Embarque'] in (FALTANTES):
        print('----------------------------------------------------------------')
        print('Embarque: ',salida.iloc[i]['Embarque'])
        url = salida.iloc[i]['URL']
        print(f'Archivo: ',salida.iloc[i]['FechaHora'],url)
        # Procesar cada Embarque
        resultado = procesar_entregas_desde_url(url, modo='imprimir')
                

In [33]:
# RUTINA PARA GRABAR en Archivo las SALIDAS 

with open('./data/resultados.txt', 'a', encoding="utf-8") as f:
    f.write(f"PROCESO DE INTEFACES desde resultado {fecha_inicio}" + '\n')
    f.close()

for i in range(len(salida)):
    if salida.iloc[i]['Embarque'] in (FALTANTES):
        with open('resultados.txt', 'a', encoding="utf-8") as f:
            f.write('\n----------------------------------------------------------------\n')
            f.write(f"Embarque: '{salida.iloc[i]['Embarque']}")
            url = salida.iloc[i]['URL']
            f.write(f"Archivo:  {salida.iloc[i]['FechaHora']} - {url}")
            f.close()
        # Procesar cada Embarque
        resultado = procesar_entregas_desde_url(url, modo='almacenar')

### Otras Alternativas para Analizar JSON Complejos

https://www.ionos.es/digitalguide/paginas-web/desarrollo-web/python-jsonpath/#:~:text=Para%20que%20Python%20pueda%20leer,%E2%80%9D%20o%20%E2%80%9Cload()%E2%80%9D.



In [34]:
salida.head(10)

In [35]:
# EXPORTAR los datos a EXCEL 
salida.to_excel("./salida.xlsx")

## 1) Generar Conexión con SQL SERVER

In [ ]:
# CONECTAR con SERVIDOR SQL SERVER

import pyodbc
from dotenv import dotenv_values

secrets = dotenv_values(".env")
local_secrets = dotenv_values(".env.dev")

SERVER = secrets["SERVIDOR"]
DATABASE = secrets["DATABASE"]
USERNAME = secrets["USUARIO"]
PASSWORD = secrets["CONTRASENA"]
PORT = secrets["PUERTO"]
DRIVER = secrets["DRIVER"]
OPTIONS= secrets["OPTION"]

connectionString = f'DRIVER={DRIVER};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD};{OPTIONS}'
print(connectionString)
conn = pyodbc.connect(connectionString) 

#    DRIVER={ODBC Driver 18 for SQL Server};SERVER=213.134.40.73,9595;DATABASE=seamtrack;UID=eduardo.ettlin@vigiloo.com.ar;PWD=Aladelta10$;encrypt=no
#    DRIVER={ODBC Driver 18 for SQL Server};SERVER=username;DATABASE=secret;UID=eduar;PWD=Aladelta10$;encrypt=no

## 2) Extraerlos Datos procesados que están en SQL SERVER

In [34]:
# Armar la Consulta con sus PARAMETROS

SQL_QUERY = f"""
SELECT   DISTINCT     
A.[RefId] as EMBARQUE
,D.RefId as DELIVERY
,A.[Date] as FECHA
,D.[PurchaseOrderIdRef] as PEDIDO
,D.[DestinationCode]
,D.[DestinationName]
,E.[MaterialCode]
,E.[MaterialName]
,E.[Batch]
,STR(E.[Qty],6) as QTY
,A.[LogisticGroupId]

FROM [seamtrack].[vigiloo].[Shipment] A
LEFT JOIN [seamtrack].[vigiloo].[ShipmentDestination] B
ON B.[ShipmentId] = A.Id
LEFT JOIN [seamtrack].[vigiloo].[Delivery] D
ON D.[ShipmentId] = A.Id
LEFT JOIN [seamtrack].[vigiloo].[DeliveryItem] E
ON E.[DeliveryId] = D.Id
    
WHERE A.[RefId] IN {REGISTROS}
ORDER BY A.RefId,D.RefId, D.[DestinationCode],E.[MaterialCode];
"""


In [ ]:
print(SQL_QUERY)

In [ ]:
# Ejecutar la Consulta
cursor = conn.cursor()
cursor.execute(SQL_QUERY)


In [37]:
records = cursor.fetchall()

In [ ]:
# IMPRIMIR Resultados de la Consulta
for r in records:
    print(f"{r.EMBARQUE}\t{r.DELIVERY}\t{r.FECHA}\t{r.DestinationCode}\t{r.DestinationName}\t{r.MaterialName}\t{r.Batch}\t{r.QTY}\t{r.LogisticGroupId}")

In [39]:
# Generar DATAFRAME con los resultados de las consultas
df = pd.DataFrame((tuple(t) for t in records)) 
df.columns = ['EMBARQUE','DELIVERY','FECHA','PEDIDO','DestinationCode','DestinationName','MaterialCode','MaterialName','Batch','QTY','LogisticGroupId']

In [40]:
# Eliminar Registros con datos Nulos
df.dropna(subset=['EMBARQUE','DELIVERY'], inplace=True)

In [41]:
# Adecuar el Grupo Logístico
df["LogisticGroupId"] = df["LogisticGroupId"].replace(
    { "75247427-34FD-41A3-B7E0-F0B8312EADA8": "Monsanto", "F21EE1F9-4823-42B4-AD40-8C81F2847818": "BAYER" })

In [ ]:
df.head(20)

In [ ]:
df.info()

In [61]:
# EXPORTAR DATOS EN FORMATO EXCEL

df.to_excel("./data/SQLServer.xlsx")

## CONTROL COMPLETO de TODAS LAS INTERFACES

#### Verificar si todo lo que está en LoadUpload EXISTE en las bases de VIGILOO

In [ ]:
print(len(REGISTROS))

In [46]:
# Armar la Consulta con sus PARAMETROS

SQL_QUERY = f"""
SELECT DISTINCT     
       A.[RefId] as EMBARQUE
FROM [seamtrack].[vigiloo].[Shipment] A
"""

In [ ]:
# Ejecutar la Consulta
cursor = conn.cursor()
cursor.execute(SQL_QUERY)

In [48]:
records = cursor.fetchall()

In [49]:
# Generar DATAFRAME con los resultados de las consultas
sqldf = pd.DataFrame((tuple(t) for t in records)) 
sqldf.columns = ['EMBARQUE']

In [ ]:
sqldf.head()

### Comparación de DATAFRAMES

#### 1) Archivos en Interfaces que NO ESTAN en la base de Datos. 

In [51]:
# UTILIZANDO LEFT JOIN
# Realizar un merge left con un indicador que muestre de dónde vienen los datos
merged_df = pd.merge(df, sqldf, on='EMBARQUE', how='left', indicator=True)

# Filtrar los registros que solo existen en el DataFrame 'df'
result_df = merged_df[merged_df['_merge'] == 'left_only']

# Seleccionar solo las columnas del DataFrame original 'df'
result_df = result_df[df.columns]


In [ ]:
print("Cantidad de INTEFACES NO PROCESADAS: ",len(result_df))

In [ ]:
result_df.head()

In [54]:
# Crear una serie booleana que indica si cada valor de 'EMBARQUE' en 'df' está también en 'sqldf'
mask = df['EMBARQUE'].isin(sqldf['EMBARQUE'])

# Usar la negación de la máscara para filtrar registros
result_df = df[~mask]



#### 2) Embarque que estan en la Base de Datos y NO ESTAN en Interfaces

In [55]:
# UTILIZANDO JOIN RIGHT
# Realizar un merge right con un indicador que muestre de dónde vienen los datos
merged_df = pd.merge(df, sqldf, on='EMBARQUE', how='right', indicator=True)

# Filtrar los registros que solo existen en el DataFrame 'sqldf'
result_df = merged_df[merged_df['_merge'] == 'right_only']

# Seleccionar solo las columnas del DataFrame original 'sqldf'
result_df = result_df[sqldf.columns]

In [56]:
# ALTERNATIVA --> Verificando si está contenido
# Crear una serie booleana que indica si cada valor de 'EMBARQUE' en 'sqldf' está también en 'df'
mask = sqldf['EMBARQUE'].isin(df['EMBARQUE'])

# Usar la negación de la máscara para filtrar registros
result_df = sqldf[~mask]


In [ ]:
print("Cantidad de EMBARQUE en la BD que no estan en INTEFACES: ",len(result_df))

In [ ]:
result_df.head()